# apply 메서드 활용
    apply 메서드 기초
    apply 메서드 고급

##### 대용량 데이터 처리시, apply메서드가 for문보다 더 빠름

### 10-2 apply 메서드 - 기초

##### 시리즈와 apply메서드 : df[컬럼명].apply(함수, 인자), 시리즈.apply(함수, 인자)

In [2]:
def my_sq(x):
    return x**2

def my_exp(x, n):
    return x**n

import pandas as pd

df = pd.DataFrame({'a':[10, 20, 30], 'b':[20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [3]:
print(df['a']**2)

0    100
1    400
2    900
Name: a, dtype: int64


In [5]:
#정의한 함수(my_sq)가 인자 1개를 받도록 구성된 경우 > 인자값 생략
    #df['a']가 첫번째 인자값으로 자동으로 넘어가기 때문, 즉 시리즈 값이 첫번째 인자값으로 넘어감
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
#인자값 2개 전달해야 할 경우?
ex = df['a'].apply(my_exp, n=2)
print(ex)
print()
    #첫번째 인자값(x)은 생략, 두번째 인자값(n)은 지정
    
ex = df['a'].apply(my_exp, n=3)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64

0     1000
1     8000
2    27000
Name: a, dtype: int64


##### 데이터프레임과 apply메서드 : df.apply(함수, axis=0,1)

In [11]:
df = pd.DataFrame({'a':[10,20,30], 'b':[20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [16]:
def print_me(x):
    print(x)

print(df.apply(print_me, axis=0))
print()
print(df.apply(print_me, axis=1))
#axis 값에 따라 함수를 열/행 방향 으로 적용할 수 있음
    #axit=0 : 열 방향 함수 적용(default)
    #axit=1 : 행 방향 함수 적용
    

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object

a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64
0    None
1    None
2    None
dtype: object


In [17]:
#인자 3개 입력받는 경우?
def avg_3(x, y, z):
    return (x + y + z) / 3

print(df.apply(avg_3))
    #인자 3개중, x만 열단위 데이터로 입력받았고 인자 2개가 더 필요하다는 오류 발생

TypeError: avg_3() missing 2 required positional arguments: 'y' and 'z'

In [18]:
#인자 3개 함수내부에서 지정
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x + y + z) / 3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [20]:
print(df)
print(df.shape) #(3행, 2열)

#df의 행이 3개임을 알고 아래의 함수 작성
    #일반적으로 아래처럼 함수 작성함
def avg_3_apply(col):
    sum = 0
    for item in col:
        sum += item
    return sum / df.shape[0]

    a   b
0  10  20
1  20  30
2  30  40
(3, 2)


In [21]:
#위 함수 응용, 행함수(axis = 1)
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum += item
    return sum / df.shape[1]    #행 단위 함수니까 요소가 a,b 2개, shape[1]로 열 갯수만큼 나눠줌

print(df.apply(avg_2_apply, axis = 1))

0    15.0
1    25.0
2    35.0
dtype: float64


### 10-3 apply 메서드 - 고급

##### 데이터프레임 누락값 처리 > apply 메서드 사용

###### 누락값처리 - 열방향

In [23]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


In [24]:
#누락값 개수 반환 함수 정의
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [25]:
#누락값 비율 계산함수 정의
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [27]:
#번외) 정상값(1-누락값) 비율 함수
def prop_complete(vec):
    return 1 - prop_missing(vec)

pcom_col = titanic.apply(prop_complete)
print(pcom_col)

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive          1.000000
alone          1.000000
dtype: float64


###### 누락값처리 - 행방향

In [29]:
cmis_row = titanic.apply(count_missing, axis = 1)
pmis_row = titanic.apply(prop_missing, axis = 1)
pcom_row = titanic.apply(prop_complete, axis = 1)

print(cmis_row.head())
print()
print(pmis_row.head())
print()
print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [30]:
#누락값 개수 titanic df에 추가
titanic['num_missing'] = titanic.apply(count_missing, axis = 1)
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [31]:
#각 행데이터 누락값을 추가함으로써, 누락값 있는 데이터만 추출 가능해짐
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex  age  sibsp  parch     fare embarked   class  \
563         0       3    male  NaN      0      0   8.0500        S   Third   
358         1       3  female  NaN      0      0   7.8792        Q   Third   
602         0       1    male  NaN      0      0  42.4000        S   First   
674         0       2    male  NaN      0      0   0.0000        S  Second   
431         1       3  female  NaN      1      0  16.1000        S   Third   
538         0       3    male  NaN      0      0  14.5000        S   Third   
680         0       3  female  NaN      0      0   8.1375        Q   Third   
301         1       3    male  NaN      2      0  23.2500        Q   Third   
451         0       3    male  NaN      1      0  19.9667        S   Third   
778         0       3    male  NaN      0      0   7.7375        Q   Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
563    man        True  NaN  Southampton    no   True            2  
358